In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
import sys
sys.path.append("/dsmlp/home-fs04/19/019/riling/scalable_rl_portfolio_management")


In [4]:
from tics.tic_config import tics_176, tics_grouped
from agent.data_downloader import short_name_sha256

In [5]:
tics = tics_176
train_start_date= '2009-01-01'
train_end_date= '2020-07-01'
test_start_date= '2020-07-01'
test_end_date= '2021-10-01'

In [6]:
start = train_start_date
end = test_end_date

In [97]:
def compute_portfolio_ohlcv_with_values(
    tics_df: pd.DataFrame,
    weights_df: pd.DataFrame,
    value_df: pd.DataFrame,
    name: str
) -> pd.DataFrame:
    """
    计算组合 OHLCV：
      - 跳过首日
      - open 使用前一日收盘市值（cap_prev）
      - 持仓股数 = 当日开盘可投资资本 / 当日开盘价
      - high = 现金 + Σ(shares * 当日high)
      - low  = 现金 + Σ(shares * 当日low)
      - close = 当日收盘市值（account_value）
      - volume = Σ(shares * 当日volume)
    """
    # 重命名并转换日期列
    weights_df = weights_df.rename(columns={'dates':'date'})
    for df in (tics_df, weights_df, value_df):
        df['date'] = pd.to_datetime(df['date'])

    # 准备账户价值和前一日资本
    val = (
        value_df[['date','account_value']]
        .drop_duplicates()
        .sort_values('date')
        .set_index('date')
    )
    val['cap_prev'] = val['account_value'].shift(1)

    # 解析权重数组
    def parse_weights(s: str):
        return np.array([float(x) for x in s.strip('[]').split()])

    wdf = (
        weights_df
        .assign(w_arr=lambda d: d['weights'].apply(parse_weights))
        .sort_values('date')
        .reset_index(drop=True)
    )

    records = []
    # 遍历日期，跳过首日和末日
    for i in range(1, len(wdf)-1):
        date = wdf.at[i, 'date']
        cap_prev = val.at[date, 'cap_prev']
        if pd.isna(cap_prev):
            continue

        # 使用前一日现金权重计算现金资本
        prev_w_all = wdf.at[i-1, 'w_arr']
        prev_cash = prev_w_all[0]
        cash_cap = cap_prev * prev_cash
        stock_cap = cap_prev - cash_cap

        # 当日行情，按ticker排序
        day = (
            tics_df[tics_df['date']==date]
            .sort_values('tic')
            .reset_index(drop=True)
        )
        opens  = day['open'].values
        highs  = day['high'].values
        lows   = day['low'].values
        vols   = day['volume'].values

        # 计算持仓股数：全部股票资本按当日开盘執行
        weights_stock = prev_w_all[1:]
        shares = (weights_stock * stock_cap) / opens

        # 计算 OHLCV
        open_port   = cap_prev
        high_port   = cash_cap + shares.dot(highs)
        low_port    = cash_cap + shares.dot(lows)
        close_port  = val.at[date, 'account_value']
        volume_port = shares.dot(vols)

        records.append({
            'date':   date,
            'open':   open_port,
            'high':   high_port,
            'low':    low_port,
            'close':  close_port,
            'volume': volume_port,
            'tic':    name,
            'day':    day['day'].iloc[0]
        })

    return pd.DataFrame(records)

In [75]:
# tics_data_file = Path("data") / 'sub' / f"{short_name_sha256('_'.join(tics_grouped[0]))}_{start}_{end}.csv"
# weights_data_file = Path("weights") / f"{short_name_sha256('_'.join(tics_grouped[0]))}_{start}_{end}.csv"

In [76]:
# tics_df = pd.read_csv(tics_data_file) 
# weights_df = pd.read_csv(weights_data_file)

In [77]:
# tics_df.head()

In [78]:
# weights_df.head()

In [79]:
# df = compute_weighted_df_simple(tics_df, weights_df)

In [98]:
dfs = []
tics = []
for i in range(6):
    tics += tics_grouped[i]
    name = short_name_sha256('_'.join(tics_grouped[i]))
    tics_data_file = Path("data") / 'sub' / f"{name}_{start}_{end}.csv"
    weights_data_file = Path("weights") / f"{name}_{start}_{end}.csv"
    vaulue_data_file = Path("acount_value") / f"{name}_{start}_{end}.csv"
    tics_df = pd.read_csv(tics_data_file).drop_duplicates()
    weights_df = pd.read_csv(weights_data_file).drop_duplicates()
    value_df = pd.read_csv(vaulue_data_file).drop_duplicates()
    dfs.append(compute_portfolio_ohlcv_with_values(tics_df, weights_df, value_df, name))

In [99]:
tics = []
for group in tics_grouped:
    tics.append(short_name_sha256('_'.join(tics)))
tics

['e3b0c44298fc1c14',
 '8e2d0c61b0acc423',
 '56b6ea2e42d34e3e',
 '1e11175f3bd186fb',
 'd725558425436f33',
 '51e648daa4892612']

In [107]:
pd.concat(dfs).to_csv(Path("data") / "weighted_avg" / f"{short_name_sha256('_'.join(tics))}_{start}_{end}.csv", index=False)

In [108]:
short_name_sha256('_'.join(tics))

'4d3dda9bdb14d5fc'

In [109]:
df = pd.read_csv(Path("data") / "weighted_avg" / f"{short_name_sha256('_'.join(tics))}_{start}_{end}.csv")
df.head()

,date,open,high,low,close,volume,tic,day
0,2009-03-18,1037264.40,1.043336e+06,9.808129e+05,1056196.00,7.279017e+11,8190e4275b4db67d,2
1,2009-03-19,1056196.00,1.030768e+06,9.807336e+05,1040738.56,5.913058e+11,8190e4275b4db67d,3
2,2009-03-20,1040738.56,1.015272e+06,9.575858e+05,1018712.40,6.362306e+11,8190e4275b4db67d,4
3,2009-03-23,1018712.40,1.033204e+06,9.641281e+05,1098340.20,5.757913e+11,8190e4275b4db67d,0
4,2009-03-24,1098340.20,1.088860e+06,1.034994e+06,1080273.20,6.759440e+11,8190e4275b4db67d,1


In [110]:
(df['high'] >= df['close']).sum()

1494

In [37]:
df = pd.read_csv(Path("data") / "weighted_avg" / f"{short_name_sha256('_'.join(tics))}_{start}_{end}.csv")
df.head()

,date,open,high,low,close,volume,tic,day
0,2009-03-17,NaN,NaN,NaN,NaN,NaN,8190e4275b4db67d,1
1,2009-03-18,1037264.40,1.082763e+06,1.017705e+06,815405.919930,7.654346e+11,8190e4275b4db67d,2
2,2009-03-19,1056196.00,1.070319e+06,1.019540e+06,787467.878847,5.899449e+11,8190e4275b4db67d,3
3,2009-03-20,1040738.56,1.055686e+06,9.955721e+05,768107.717684,6.454607e+11,8190e4275b4db67d,4
4,2009-03-23,1018712.40,1.075351e+06,1.001907e+06,814658.470357,6.492611e+11,8190e4275b4db67d,0
